In [ ]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import torch
RESOLUTION = 331

In [ ]:
# Shape:  (89682, 331, 331, 3)
# 8种属性总的means: [0.63894498, 0.5993557, 0.587506]
# 8种属性总的stdevs: [0.28748438, 0.29435793, 0.29647669]
# transforms.Normalize(mean = [0.63894498, 0.5993557, 0.587506], std = [0.28748438, 0.29435793, 0.29647669])

In [ ]:
#计算每个属性的mean和std
classes = [ 'collar_design_labels','neckline_design_labels','skirt_length_labels', 
             'sleeve_length_labels', 'neck_design_labels', 'lapel_design_labels', 
             'pant_length_labels','coat_length_labels']  
for cls in classes:
    df_train = pd.read_csv('../train/Annotations/{0}.csv'.format(cls))
    df_load = df_train.copy()
    df_load.columns = ['image_id', 'class', 'label']
    df_load.reset_index(inplace= True,drop=True)
    
    n = len(df_load)
    X = np.zeros((n, RESOLUTION, RESOLUTION, 3), dtype='float32')
    print(cls,'该属性样本数:{}'.format(n))
    for i in range(n):
        X[i] = cv2.resize(cv2.imread('../train/{0}'.format(df_load['image_id'][i])), (RESOLUTION, RESOLUTION))/255.

    print(cls,"DataShape: ", X.shape)

    means =  []
    stdevs = []
    for i in range(3):
        pixels = X[:,:,:,i].ravel()
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))
    print(cls,"该属性means: {}".format(means[::-1]))
    print(cls,"该属性stdevs: {}".format(stdevs[::-1]))
    print(cls,':','transforms.Normalize(mean = {}, std = {})'.format(means[::-1], stdevs[::-1]))
    print()

In [ ]:
#计算所有属性的平均means和std
# Ideally we shouldn't be resizing but I'm lacking memory
df_train = pd.read_csv('../train/Annotations/train.csv')
df_load = df_train.copy()
df_load.columns = ['image_id', 'class', 'label']
df_load.reset_index(inplace= True,drop=True)

n = len(df_load)
X = np.zeros((n, RESOLUTION, RESOLUTION, 3), dtype='float32')


for i in range(n):
    X[i] = cv2.resize(cv2.imread('../train/{0}'.format(df_load['image_id'][i])), (RESOLUTION, RESOLUTION))/255.
    #if i == 100:
    #    print(X[i])
#train_images = df_load['image_id'].tolist()
# for file in tqdm(train_images, miniters=256):
#     #print(file)
#     train_img_name = os.path.join('../train/',file)
#     img = cv2.imread(train_img_name)
#     data.append(cv2.resize(img,(RESOLUTION,RESOLUTION)))
#data = np.array(data, np.float32) / 255 # Must use float32 at least otherwise we get over float16 limits
#XX = torch.from_numpy(X)
means =  []
stdevs = []
for i in range(3):
    pixels = X[:,:,:,i].ravel()
    means.append(np.mean(pixels))
    stdevs.append(np.std(pixels))
print("Shape: ", X.shape)
print("8种属性总的means: {}".format(means[::-1])) #RGB
print("8种属性总的stdevs: {}".format(stdevs[::-1]))#RGB
print('transforms.Normalize(mean = {}, std = {})'.format(means[::-1], stdevs[::-1]))
